In [ ]:
## Import all the possible libraries which all needed. 

import selenium
import requests
import pandas as pd
import urllib.request
import re
import os 
from tqdm import tqdm
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options

### 1. Write a python program which searches all the product under a particular product from `www.amazon.in`. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [ ]:
# get the access from chrome driver

driver = webdriver.Chrome()

In [ ]:
#website url

driver.get("https://www.amazon.in/")

In [ ]:
#search bar selection

search_bar=driver.find_element(By.ID,"twotabsearchtextbox")

In [ ]:
#take input from user: Note press enter after entering the input to get out of loop

print("Search Product you like ")
search_for=input()

In [ ]:
#search the input 

search_bar.clear()
search_bar.send_keys(search_for)

In [ ]:
#click on search button icon

search_button=driver.find_element(By.ID,"nav-search-submit-button")
search_button.click()

###  2. scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. 

In [ ]:
#fetch the urls of all products 

start_page = 0
end_page = 2
urls = []
for page in range(start_page,end_page+1):
    try:
        page_urls = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')
        
        # appending all the urls on current page to urls list
        for url in page_urls:
            url = url.get_attribute('href')     
            if url[0:4]=='http':                # Checking if the scraped data is a valid url or not
                urls.append(url)                
        print("Product urls of page {} has been scraped.".format(page+1))
        
        # Moving to next page
        nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')      # Locating the next_button which is active
        if nxt_button.text == 'Next→':                                            # Checking if the button located is next button
            nxt_button.click()                                                    
            time.sleep(5)                                                         
            
        # If the current active button is not next button, the we will check if the next button is inactive or not    
        elif driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').text == 'Next→':    
            print("No new page Found so, Breaking the loop")  
            break
            
    except StaleElementReferenceException as e:             # Handling StaleElement Exception   
        print("Stale Exception")
        next_page = nxt_button.get_attribute('href')        
        driver.get(next_page)                               

In [ ]:
#check if all products from 1st 3 pages are scraped
len(urls)

In [ ]:
## create the list for store the data
brand_list=[]
title_list=[]
rating_list=[]
no_rating_list=[]
price_list=[]
return_list=[]
expected_delivery_list=[]
availability_list=[]
details_list=[]
img_url_list=[]
     
#go to each items url to fetch data   
for a in urls:
    driver.get(a)
    
    # brand
    try:
        brand=driver.find_element(By.XPATH,"//table[@class='a-keyvalue prodDetTable']/tbody/tr[1]/td")
        brand_list.append(brand.text)
    except NoSuchElementException as e:
        brand_list.append("--")
    
    # title 
    try:
        title=driver.find_element(By.XPATH,"//h1[@class='a-size-large a-spacing-none']")
        title_list.append(title.text)
    except NoSuchElementException as e:
        title_list.append("--")
        
    # ratings
    try:
        rating=driver.find_element(By.XPATH,"//*[@id='productDetails_detailBullets_sections1']/tbody/tr[2]/td")
        rating_list.append(rating.text.split("\n")[-1])
    except NoSuchElementException as e:
        rating_list.append("--")
        
    # number of ratings
    try:
        no_rating=driver.find_element(By.ID,"averageCustomerReviews")
        no_rating_list.append(no_rating.text)
    except NoSuchElementException as e:
        no_rating_list.append("--")
        
    # price
    try:
        price=driver.find_element(By.XPATH,"//table[@class='a-lineitem']/tbody/tr[2]/td[2]/span")
        price_list.append(price.text)
    except NoSuchElementException as e:
        price_list.append("--")  
        
    # return
    try:
        returns=driver.find_element(By.XPATH,"//div[@class='a-row icon-farm-wrapper']/div[2]/span/div[2]/a")
        return_list.append(returns.text)
    except NoSuchElementException as e:
        return_list.append("--")
        
    # expected_delivery
    try:
        expected_delivery=driver.find_element(By.XPATH,"//span[@class='a-color-base a-text-bold']")
        expected_delivery_list.append(expected_delivery.text)
    except NoSuchElementException as e:
        expected_delivery_list.append("--")
    
        
    # availability
    try:
        availability=driver.find_element(By.XPATH,"//span[@class='a-size-medium a-color-success']")
        availability_list.append(availability.text)
    except NoSuchElementException as e:
        availability_list.append("--")   
        
    #other details
    try:
        details=driver.find_element(By.XPATH,"//div[@class='a-row a-spacing-top-base']")
        details_list.append(details.text.replace("\n"," "))
    except NoSuchElementException as e:
        details_list.append("--")
        
    #product image url
    try:
        img_url=driver.find_element(By.XPATH,"//div[@class='imgTagWrapper']/img")
        img_url_list.append(img_url.get_attribute("src"))
    except NoSuchElementException as e:
        img_url_list.append("--")
        


In [ ]:
print(brand_list[0:2])
print(title_list[0:2])
print(rating_list[0:2])
print(no_rating_list[0:2])
print(price_list[0:2])
print(return_list[0:2])
print(expected_delivery_list[0:2])
print(availability_list[0:2])
print(details_list[0:1])
print(img_url_list[0:2])

In [ ]:
#check if the length of all lists are equal
print(len(brand_list))
print(len(title_list))
print(len(rating_list))
print(len(no_rating_list))
print(len(price_list))
print(len(return_list))
print(len(expected_delivery_list))
print(len(availability_list))
print(len(details_list))
print(len(img_url_list))

In [ ]:
#creating data frame
df=pd.DataFrame({})
df['Brand']=brand_list
df['Name of the product']=title_list
df["Ratings"]=rating_list
df["No of ratings"]=no_rating_list
df['Price']=price_list
df['Return/Exchange']=return_list
df['Expected Delivery']=expected_delivery_list
df['Availability']=availability_list
df['Other details']=details_list
df['Product Urls']=img_url_list

df

### 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [ ]:
#connect to driver
driver=webdriver.Chrome()
driver.get('https://images.google.com/')


In [ ]:
# Finding the search bar using it's xpath

search_bar = driver.find_element(By.ID,"APjFqb")    

# Inputing "banana" keyword to search rock images
search_bar.send_keys("fruits")       

# Finding the xpath of search button
search_button = driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button")    

 # Clicking the search button
search_button.click()       

In [ ]:
# 50 time we scroll down by 100 in order to generate more images on the website

driver.execute_script("window.scrollBy(0,100)")

In [ ]:
## Fetch images by using XPATH

images = driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
len(img_urls)

In [ ]:
## using for loop for save the continue image data in the system folder

for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:\Internship _Projects_DATA\images"+str(i)+".jpg", "wb")
    file.write(response.content)

In [ ]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//div[@class="dGWkFc"]/input')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cars' in search bar
Search.send_keys('Cars')

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'rCGXm')

# Clicking search button
Search_button.click()

In [ ]:
# 50 time we scroll down by 200 in order to generate more images on the website

driver.execute_script("window.scrollBy(0,200)","")

In [ ]:
URL=[]
images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)
            
len(URL)

In [ ]:
## using for loop for save the continue image data in the system folder

images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:\Internship _Projects_DATA\images" +str(i)+ ".jpg", "wb")
    file.write(response.content)

In [ ]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//div[@class="dGWkFc"]/input')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cars' in search bar
Search.send_keys('Machine Learning')

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'rCGXm')

# Clicking search button
Search_button.click()

In [ ]:
driver.execute_script("window.scrollBy(0,100)","")

In [ ]:
URL=[]
images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [ ]:
len(URL)

In [ ]:
## using for loop for save the continue image data in the system folder

images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:\Internship _Projects_DATA\images" +str(i)+ ".jpg", "wb")
    file.write(response.content)

In [ ]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//div[@class="dGWkFc"]/input')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cars' in search bar
Search.send_keys('Guitars')

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'rCGXm')

# Clicking search button
Search_button.click()

In [ ]:
driver.execute_script("window.scrollBy(0,100)","")

In [ ]:
URL=[]
images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [ ]:
len(URL)

In [ ]:
## using for loop for save the continue image data in the system folder

images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:\Internship _Projects_DATA\images" +str(i)+ ".jpg", "wb")
    file.write(response.content)

In [ ]:
# Finding Search bar website by class
Search=driver.find_element(By.XPATH,'//div[@class="dGWkFc"]/input')

# Clearing any previous input in search bar
Search.clear()

# Feeding input 'Cars' in search bar
Search.send_keys('Cakes')

# Finding Search button for clicking through class name
Search_button=driver.find_element(By.CLASS_NAME,'rCGXm')

# Clicking search button
Search_button.click()

In [ ]:
driver.execute_script("window.scrollBy(0,100)","")

In [ ]:
URL=[]
images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            URL.append(source)

In [ ]:
len(URL)

In [ ]:
## using for loop for save the continue image data in the system folder

images= driver.find_elements(By.XPATH,'//div[@class="fR600b islir"]/img')

img_urls = []
img_data = []
for image in images:
    source= image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
            
for i in range(len(img_urls)):
    if i >= 100:
        break
    print("Downloading {0} of {1} images" .format(i, 100))
    response= requests.get(img_urls[i])
    file = open("D:\Internship _Projects_DATA\images" +str(i)+ ".jpg", "wb")
    file.write(response.content)

### 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on `www.flipkart.com`

#### and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [ ]:
#connect to driver
driver=webdriver.Chrome()

In [ ]:
## get to the flipkart webpage

url4="https://www.flipkart.com/search?q=smartphone&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
driver.get(url4)

In [ ]:
## ## create the list for store the data

Brand_Name=[]
Colour=[]                    ## Color is not available seprately it is with model and phone name
Storage_RAM_ROM=[]
P_F_Camera=[]
Display_size_Resolution=[]
Battery=[]
Price=[]
Product_URL=[]               

In [ ]:
## Brand name

BName=driver.find_elements(By.XPATH,"//div[@class='KzDlHZ']")
for i in BName:
    if i.text is None :
        Brand_Name.append("--") 
    else:
        Brand_Name.append(i.text)
print(len(Brand_Name),Brand_Name)

In [ ]:
## RAM and ROM Storage

ram=driver.find_elements(By.XPATH,"//ul[@class='G4BRas']/li[1]")
for i in ram:
    if i.text is None :
        Storage_RAM_ROM.append("--") 
    else:
        Storage_RAM_ROM.append(i.text.split())
print(len(Storage_RAM_ROM),Storage_RAM_ROM)

In [ ]:
## Primary and Secodry Camera

PC=driver.find_elements(By.XPATH,"//ul[@class='G4BRas']/li[3]")
for i in PC:
    if i.text is None :
        P_F_Camera.append("--") 
    else:
        P_F_Camera.append(i.text)
print(len(P_F_Camera),P_F_Camera)

In [ ]:
## Display size

DS=driver.find_elements(By.XPATH,"//ul[@class='G4BRas']/li[2]")
for i in DS:
    if i.text is None :
        Display_size_Resolution.append("--") 
    else:
        Display_size_Resolution.append(i.text)
print(len(Display_size_Resolution),Display_size_Resolution)

In [ ]:
## Battery Capacity

B=driver.find_elements(By.XPATH,"//ul[@class='G4BRas']//li[4]")
for i in B:
    if i.text is None :
        Battery.append("--") 
    else:
        Battery.append(i.text)
print(len(Battery),Battery)

In [ ]:
## Price

price=driver.find_elements(By.XPATH,"//div[@class='Nx9bqj _4b5DiR']")
for i in price:
    if i.text is None :
        Price.append("--") 
    else:
        Price.append(i.text)
print(len(Price),Price)

In [ ]:
## Product URL

product_url=driver.find_elements(By.XPATH,"//div[@class='tUxRFH']/a")
for i in product_url:
    if i.text is None :
        Product_URL.append("--") 
    else:
        Product_URL.append(i.text)
print(len(Product_URL),Product_URL)




In [ ]:
## Create DataFrame

df=pd.DataFrame([])
df['Brand_Name & Phone_Name']=Brand_Name[0:24]
df['RAM & ROM Storage']=Storage_RAM_ROM[0:24]
df['Camera']=P_F_Camera[0:24]
df['Display_size']=Display_size_Resolution[0:24]
df['Battery_Capacity']=Battery[0:24]
df['Price']=Price[0:24]
df['Product_url']=Product_url[0:24]


In [ ]:
df

In [ ]:
## Convert the dataframe to CSV file

df.to_csv("flipkart_smartphones_data.csv")

### Q.5 ) Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.


In [ ]:
#connect to driver
driver=webdriver.Chrome()

In [ ]:
# opening google maps
driver.get("https://www.google.co.in/maps")
time.sleep(3)

# Enter city to be searched
city = input('Enter City Name : ')

# locating search bar
search = driver.find_element(By.ID,"searchboxinput")  

 # clearing search bar
search.clear()                                                            
time.sleep(2)

# entering values in search bar
search.send_keys(city)              

# locating search button
button = driver.find_element(By.ID,"searchbox-searchbutton")

# clicking search button
button.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

### 6. Write a program to scrap all the available details of best gaming laptops from `digit.in`

In [ ]:
#connect to driver
driver=webdriver.Chrome()

In [ ]:
# visit the web page
url="https://www.digit.in/top-products/best-gaming-laptops-40.html"

In [ ]:
# Get the webpage

driver.get(url)

In [ ]:
## create the list for store the data

Brands=[]
Products_Description=[]
Specification=[]
Price=[]

In [ ]:
## brand
br=driver.find_elements(By.XPATH,"//div[@class='rh_gr_top_middle mb10 colored_rate_bar']/h3/a")
len(br)


In [ ]:
## run for loop for getting brand name data

for i in br:
   
    Brands.append(str(i.text).replace("\n",""))
Brands

In [ ]:
## Discription

discription=driver.find_elements(By.XPATH,"//div[@class='woo_code_zone_loop clearbox']")
len(discription)

In [ ]:
## run for loop for getting discription data

for i in discription:
   
    Specification.append(str(i.text).replace("\n",""))
Specification

In [ ]:
## product

prod_disc=driver.find_elements(By.TAG_NAME,"p")
len(prod_disc)

In [ ]:
## run for loop for getting Product discription data

for i in prod_disc:
   
    Products_Description.append(str(i.text).replace("\n"," "))
Products_Description

In [ ]:
## price

pri=driver.find_elements(By.XPATH,"//div[@class='cegg-price-row']")
len(pri)

In [ ]:
## run for loop for getting price data

for i in pri:
   
    Price.append(str(i.text).replace("\n",""))
Price

In [ ]:
## create the list for store the data

digit_lap=pd.DataFrame([])
digit_lap['Brands']=Brands[0:10]
digit_lap['Description']=Products_Description[0:10]
digit_lap['Specification']=Specification[0:10]
digit_lap['Price']=Price[0:10]


digit_lap

### 7. Write a python program to scrape the details for all billionaires from `www.forbes.com`. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [ ]:
## get the Chrome Access

driver = webdriver.Chrome()

In [ ]:
## take the URL of the Website

url = "https://www.forbes.com/billionaires/"

In [ ]:
## get the webpage access through URL

driver.get(url)

In [ ]:
## create the list for store the data

Rank = []
Name = []
Net_Worth= []
Age = []
Citizenship = []
Source = []
Industry = []

In [ ]:
## run a for loop for scraping the Rank data

rank=driver.find_elements(By.XPATH,"//div[@class='Table_rank__X4MKf']/div")
for i in rank:
    if i.text is None :
        Rank.append("--") 
    else:
        Rank.append(i.text)
print(len(Rank),Rank)

In [ ]:
## run a for loop for scraping the Name data

name=driver.find_elements(By.XPATH,"//div[@class='Table_personName__Bus2E']")
for i in name:
    if i.text is None :
        Name.append("--") 
    else:
        Name.append(i.text)
print(len(Name),Name)

In [ ]:
## run a for loop for scraping the Net worth data

net_worth=driver.find_elements(By.XPATH,"//div[@class='Table_finalWorth__UZA6k']/span")
for i in net_worth:
    if i.text is None :
        Net_Worth.append("--") 
    else:
        Net_Worth.append(i.text)
print(len(Net_Worth),Net_Worth)

In [ ]:
## run a for loop for scraping the Age data

age=driver.find_elements(By.XPATH,"//div[@class='Table_tableRow__lF_cY']/div[4]")
for i in age:
    if i.text is None :
        Age.append("--") 
    else:
        Age.append(i.text)
print(len(Age),Age)

In [ ]:
## run a for loop for scraping the Citizenship data

citizenship=driver.find_elements(By.XPATH,"//div[@class='Table_tableRow__lF_cY']/div[5]")
for i in citizenship:
    if i.text is None :
        Citizenship.append("none") 
    else:
        Citizenship.append(i.text)
print(len(Citizenship),Citizenship)

In [ ]:
## run a for loop for scraping the Industry data

industry=driver.find_elements(By.XPATH,"//div[@class='Table_tableRow__lF_cY']/div[6]")
for i in industry:
    if i.text is None :
        Industry.append("none") 
    else:
        Industry.append(i.text)
print(len(Industry),Industry)

In [ ]:
## run a for loop for scraping the Source of wealth data

source=driver.find_elements(By.XPATH,"//div[@class='Table_tableRow__lF_cY']/div[7]")
for i in source:
    if i.text is None :
        Source.append("none") 
    else:
        Source.append(i.text)
print(len(Source),Source)

In [ ]:
#creating dataframe

forbes_list=pd.DataFrame([])
forbes_list['Rank']=Rank[0:100]
forbes_list['Name']=Name[0:100]
forbes_list['Net Worth']=Net_Worth[0:100]
forbes_list['Age']=Age[0:100]
forbes_list['Citizenship']=Citizenship[0:100]
forbes_list['Source']=Source[0:100]
forbes_list['Industry']=Industry[0:100]



In [ ]:
forbes_list

### 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [ ]:
# Connecting to webdriver
driver=webdriver.Chrome()
time.sleep(1)

# Opening url in chrome browser
url="https://www.youtube.com"
driver.get(url)
time.sleep(3)

In [ ]:
# Finding search menu by xpath
Search=driver.find_element(By.XPATH,'//input[@id="search"]') 

# Feeding input video name by user to search menu through send keys
Search.send_keys('What are the MOST LIKED Youtube Comments on Youtube?')

# Finding Search button for clicking through xpath
Search_button=driver.find_element(By.XPATH,'//button[@id="search-icon-legacy"]/yt-icon')  

# Clicking search button
Search_button.click()
time.sleep(2)

In [ ]:
# Clicking on video
link = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link.click()
time.sleep(1)

In [ ]:
# Scrolling window using ScrollBy method from 0 pixel to 1500 pixel
driver.execute_script("window.scrollBy(0,100)","")

In [ ]:
# create the lists for store the data
Comments = []
Comment_posted_ago = []
Timeline = []
Likes = []


In [ ]:
#extracting comment on video

comment = driver.find_elements(By.ID,"content-text")
time.sleep(1)
for i in tqdm(comment):
    if i.text is None:
        Comments.append("--")
    else:
        Comments.append(i.text)
time.sleep(2)

In [ ]:
# Extracting time of commenting

timeline = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
time.sleep(3)
for i in tqdm(timeline):
    if i.text is None:
        Timeline.append("-")
    else:
        Timeline.append(i.text)
for i in tqdm(range(0,len(Timeline),2)):
    Comment_posted_ago.append(Timeline[i])

In [ ]:
## verify the length of comments 

len(Comments),len(Comment_posted_ago)

In [ ]:
## Create the dataframe 

you_tube=pd.DataFrame({})
you_tube['Comments']=Comments[:500]
you_tube['Comment Posted Ago']=Comment_posted_ago[:500]

you_tube


### 9. Write a python program to scrape a data for all available Hostels from `https://www.hostelworld.com/` in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
# Connecting to webdriver
driver=webdriver.Chrome()
time.sleep(1)

# Opening url in chrome browser
url="https://www.hostelworld.com"
driver.get(url)
time.sleep(2)


In [ ]:
## place input in the searchh bar

loc=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/main/header/div/div[2]/div[1]/div[1]/div/div[1]/div[1]/div/div[2]/label/input')
loc.send_keys("london")

In [ ]:
## Click on the search bar for finalize the location

location_search = driver.find_element(By.XPATH,"//li[@class='item is-two-row']/button")
location_search.click()

In [ ]:
## now click on the search button

search_button = driver.find_element(By.XPATH,"//button[@class='large-button btn-content']") 
search_button.click()

In [ ]:
## Create the lsit for store the data

Hostel_name=[]
Distance_from_city_centre=[]
Ratings=[]
Total_reviews=[]
Overall_reviews=[]
Privates_from_price=[]
Dorms_from_price=[]
Product_url = []

# not able to find these two elements data on the same page 

# Facilities=[]
# Property_discription=[]

In [ ]:
# Scrape the data using Attibute types and store in the list
# Names

names=driver.find_elements(By.XPATH,"//div[@class='property-name']/span")
for i in names:
    if i.text is None :
        Hostel_name.append("--") 
    else:
        Hostel_name.append(i.text)
print(len(Hostel_name),Hostel_name)

In [ ]:
## Distance from the city centre

distances=driver.find_elements(By.XPATH,"//span[@class='distance-description']")
for i in distances:
    if i.text is None :
        Distance_from_city_centre.append("--") 
    else:
        Distance_from_city_centre.append(i.text)
print(len(Distance_from_city_centre),Distance_from_city_centre)

In [ ]:
## Ratings

ratings=driver.find_elements(By.XPATH,"//span[@class='number']")
for i in ratings:
    if i.text is None :
        Ratings.append("--") 
    else:
        Ratings.append(i.text)
print(len(Ratings),Ratings)

In [ ]:
## Total Review Count

total_reviews=driver.find_elements(By.XPATH,"//div[@class='review']/span")
for i in total_reviews:
    if i.text is None :
        Total_reviews.append("--") 
    else:
        Total_reviews.append(i.text)
print(len(Total_reviews),Total_reviews)

In [ ]:
## Overall review

overall_revivews=driver.find_elements(By.XPATH,"//span[@class='keyword']")
for i in overall_revivews:
    if i.text is None :
        Overall_reviews.append("--") 
    else:
        Overall_reviews.append(i.text)
print(len(Overall_reviews),Overall_reviews)

In [ ]:
## private type rent 

private_prices=driver.find_elements(By.TAG_NAME,"strong")
for i in private_prices:
    if i.text is None :
        Privates_from_price.append("--") 
    else:
        Privates_from_price.append(i.text)
print(len(Privates_from_price),Privates_from_price)

In [ ]:
## Dormatory type rent 

dorms_prices=driver.find_elements(By.CLASS_NAME,"current")
for i in dorms_prices:
    if i.text is None :
        Dorms_from_price.append("--") 
    else:
        Dorms_from_price.append(i.text)
print(len(Dorms_from_price),Dorms_from_price)

In [ ]:
## Property URL

product_urls = driver.find_elements(By.XPATH,"//div[@class='property-info-container']")
for i in product_urls:
    if i.text is None :
        Product_url.append("--") 
    else:
        Product_url.append(i.text)
print(len(Product_url),Product_url)


In [ ]:
## Create the DataFrame 

hostels=pd.DataFrame([])
hostels['Hostel_name']=Hostel_name[0:30]
hostels['Distance_from_city_centre']=Distance_from_city_centre[0:30]
hostels['Ratings']=Ratings[0:30]
hostels['Total_reviews']=Total_reviews[0:30]
hostels['Overall_reviews']=Overall_reviews[0:30]
hostels['Privates_from_price']=Privates_from_price[0:30]
hostels['Dorms_from_price']=Dorms_from_price[0:30]
hostels['Product_url']=Product_url[0:30]

#hostels['Facilities']=Facilities[0:30]
#hostels['Property_discription']=Property_discription[0:30]


In [ ]:
hostels

### End Of all Questions